# Import libraries

In [1]:
import os, glob, csv, sys, re
from os import path
from urllib.parse import urlparse
import pandas as pd
from bs4 import BeautifulSoup
from tld import get_tld

# Define directories

In [2]:
userhome = os.path.expanduser('~')
analyze_dir = userhome + r'/mining-forum/analyze/'
summ_dir = userhome + r'/mining-forum/analyze/csv_files/summarize/'

Define function to determine the directory of the dataset

In [3]:
def dir_num(x):
    if (x < 44000):
        y = 12
    elif (x < 88000):
        y = 13
    elif (x < 132000):
        y = 14
    elif (x < 176000):
        y = 15
    elif (x < 220000):
        y = 16
    elif (x < 440000):
        y = 17
    elif (x < 660000):
        y = 18
    elif (x < 880000):
        y = 19
    else:
        y = 20
    
    return y

# Open dataset of random sampled threads with purposes

In [4]:
df_threads = pd.read_csv(summ_dir + 'threads_of_randomsampling_with_purpose.csv')
df_threads

,file_number,message_id,title,author,purpose
0,154170,message #485567,Using name of source file in generation,Marco,Q&A
1,728528,message #1330113,"Another ""Ambiguous variable type defintions"" p...",Emre T,Q&A
2,628051,message #1219680,Call xtend function in java,Caroline Smith,Q&A
3,314487,message #826625,migrating to Xtext 2.3 and ITreeAppendable,Lorenzo Bettini,Q&A
4,106904,message #326841,Setting parameters via API,Eclipse User,Q&A
5,191306,message #606853,Validation Exception with Teneo 0.7.5 (April 2...,Sundeep Prakash,Q&A
6,85708,message #266027,Running eclipse-based applciation as standalone?,Eclipse User,Q&A
7,139534,message #439798,is there a preferred RCP mechanism for updatin...,Paul E. Keyser,Q&A
8,285273,message #790811,unable to launch,yasin alm,Q&A
9,123228,message #376309,oAW error :'Couldn't find operation....',Eclipse User,Q&A


In [5]:
# cols = ['file_number', 'message_id', 'title', 'author', 'purpose']
# df_QA_threads = df_threads[cols][df_threads[cols]['purpose'] == 'Q&A']
# df_QA_threads

# Extracting links

In [6]:
url_data = []
for file_num in range(0,len(df_threads)):
    fnum = df_threads.iloc[file_num][0]
    file = userhome + r'/mining-forum/data/forumfiles_' + str(dir_num(fnum)) + '/' + str(fnum)
    
    sys.stdout.write('\rExtracting file number: %i' %(file_num+1) + ' / %i' %(len(df_threads)) + ' --> Forum files : %i' %(dir_num(fnum)))
    sys.stdout.flush()

    f = open(file)
    f = f.read()

    try:
        soup = BeautifulSoup(f, 'html.parser')
        topic = soup.find('a', attrs={'class':'MsgSubText'})
        for title in topic.stripped_strings:
            title = title

        msid = []
        soup = BeautifulSoup(f, 'html.parser')
        for message_id in soup.find_all('td', attrs={'class':'MsgR1 vt al MsgSubText'}):
            for mid in message_id.find_all('span', attrs={'class':'SmallText'}):
                for msgid in mid.find_all('a'):
                    msid.append(msgid.text)
        msid = msid[0]

        data = [df_threads.iloc[file_num][0], df_threads.iloc[file_num][4], msid, title]
        soup = BeautifulSoup(f, 'html.parser')
        for userlink in soup.find_all('td', attrs={'class':'MsgSpacer'}):
            for user in userlink.find_all('td', attrs={'class':'msgud'}):
                for usr in user.find_all('a'):
                    username = usr.text
                    if username == '':
                        username = 'Eclipse User'

                    for link in userlink.find_all('span', attrs={'class':'MsgBodyText'}):
                        for atag in link.find_all('a'):
                            ss = atag.text.replace(' ','')
                            parser = []
                            try:
                                link = re.search('^https?:\/\/.*[\r\n]*[^.\\,:;]', ss)
                                link = link.group()
                                
                                try:       
                                    regpath = re.search("http.?:\/\/(\w*.+?\/\w*[a-zA-Z0-9._#:%@&?=-~]+\w*)\/?", link)
                                    regpath = regpath.group(1)
                                except:
                                    regpath = link
                                
                                domain = urlparse(link)
                                tld_dt = get_tld(link)
                                parser = [username, link, regpath, domain.netloc, tld_dt]
                                data.insert(len(data),parser)
                                
                                
                                
                            except:
                                pass

        n = 4
        link_name = []
        for i in range (0,len(data)-3):
            if data[n][1] not in link_name:
                url_data.append([data[0], data[1], data[2], data[3], data[n][0], data[n][1], data[n][2], data[n][3], data[n][4]])
                link_name.append(data[n][1])
            else:
                pass
            n = n + 1
    
    except:
        pass

print ('\nFiles extraction are finished')

Extracting file number: 384 / 384 --> Forum files : 14
Files extraction are finished


In [7]:
url_data

[[728528,
  'Q&A',
  'message #1330113',
  'Another "Ambiguous variable type defintions" problem',
  'Emre T',
  'http://stackoverflow.com/questions/19702962/eclipse-manifest-mf-plugin-dependencies-different-icons',
  'stackoverflow.com/questions',
  'stackoverflow.com',
  'com'],
 [628051,
  'Q&A',
  'message #1219680',
  'Call xtend function in java',
  'Christian Dietrich',
  'http://xtext.itemis.com',
  'http://xtext.itemis.com',
  'xtext.itemis.com',
  'com'],
 [314487,
  'Q&A',
  'message #826625',
  'migrating to Xtext 2.3 and ITreeAppendable',
  'Lorenzo Bettini',
  'http://www.lorenzobettini.it',
  'http://www.lorenzobettini.it',
  'www.lorenzobettini.it',
  'it'],
 [314487,
  'Q&A',
  'message #826625',
  'migrating to Xtext 2.3 and ITreeAppendable',
  'Lorenzo Bettini',
  'http://www.purplesucker.com',
  'http://www.purplesucker.com',
  'www.purplesucker.com',
  'com'],
 [314487,
  'Q&A',
  'message #826625',
  'migrating to Xtext 2.3 and ITreeAppendable',
  'Lorenzo Bettini

In [8]:
len(url_data)

294

In [11]:
with open (summ_dir + "links_from_randomsampled_threads.csv", "w") as mycsv:
    head = ['filenumber','purpose', 'message_id', 'title', 'author_name', 'full_link', 'link_path', 'domain', 'top_level_domain']
    writers = csv.writer(mycsv, delimiter = ',')
    writers.writerow(head)
    for item in url_data:
        writers.writerow(item)
print ("CSV file has been created")

CSV file has been created


# Categorizing

In [12]:
df_link = pd.read_csv(summ_dir + "links_from_randomsampled_threads.csv")
df_link

,filenumber,purpose,message_id,title,author_name,full_link,link_path,domain,top_level_domain
0,728528,Q&A,message #1330113,"Another ""Ambiguous variable type defintions"" p...",Emre T,http://stackoverflow.com/questions/19702962/ec...,stackoverflow.com/questions,stackoverflow.com,com
1,628051,Q&A,message #1219680,Call xtend function in java,Christian Dietrich,http://xtext.itemis.com,http://xtext.itemis.com,xtext.itemis.com,com
2,314487,Q&A,message #826625,migrating to Xtext 2.3 and ITreeAppendable,Lorenzo Bettini,http://www.lorenzobettini.it,http://www.lorenzobettini.it,www.lorenzobettini.it,it
3,314487,Q&A,message #826625,migrating to Xtext 2.3 and ITreeAppendable,Lorenzo Bettini,http://www.purplesucker.com,http://www.purplesucker.com,www.purplesucker.com,com
4,314487,Q&A,message #826625,migrating to Xtext 2.3 and ITreeAppendable,Lorenzo Bettini,http://www.myspace.com/supertrouperabba,www.myspace.com/supertrouperabba,www.myspace.com,com
5,314487,Q&A,message #826625,migrating to Xtext 2.3 and ITreeAppendable,Lorenzo Bettini,http://tronprog.blogspot.com,http://tronprog.blogspot.com,tronprog.blogspot.com,blogspot.com
6,314487,Q&A,message #826625,migrating to Xtext 2.3 and ITreeAppendable,Lorenzo Bettini,http://longlivemusic.blogspot.com,http://longlivemusic.blogspot.com,longlivemusic.blogspot.com,blogspot.com
7,314487,Q&A,message #826625,migrating to Xtext 2.3 and ITreeAppendable,Lorenzo Bettini,http://www.gnu.org/software/src-highlite,www.gnu.org/software,www.gnu.org,org
8,314487,Q&A,message #826625,migrating to Xtext 2.3 and ITreeAppendable,Lorenzo Bettini,http://www.gnu.org/software/gengetopt,www.gnu.org/software,www.gnu.org,org
9,314487,Q&A,message #826625,migrating to Xtext 2.3 and ITreeAppendable,Lorenzo Bettini,http://www.gnu.org/software/gengen,www.gnu.org/software,www.gnu.org,org


In [13]:
df_purpose = df_link.groupby(['purpose'])['purpose'].count().to_frame()
df_purpose = df_purpose.sort_values('purpose', ascending=False)
df_purpose

,purpose
purpose,
Q&A,236
Announcement,22
General discussion,16
Misuse,13
Information,7
